In [1]:
from dataclasses import dataclass
import pulp
import pandas as pd
import yaml

@dataclass
class Armor:
    poise: int
    weight: float
    name: str
    location: str = None
    _var = None

    @property
    def var(self):
        if self._var is None:
            self._var = pulp.LpVariable(self.name, 0, 1, pulp.LpInteger)
        return self._var


def get_armors(armor_type):
    df = pd.read_pickle(f"{armor_type}.pkl")
    df["Poise"] = pd.to_numeric(df["Poise"], errors='coerce')
    df["Wgt."] = pd.to_numeric(df["Wgt."], errors='coerce')
    df.fillna(0, inplace=True)
    return [Armor(name = armor["Name"], poise=armor["Poise"], weight=armor["Wgt."], location=armor["How to Acquire"] ) for armor in df.to_dict(orient='records')]

with open('armor_values.yaml') as f:
    my_armors = yaml.load(f, Loader=yaml.Loader)

def solve(weight_limit, my_armors_only = False):
    prob = pulp.LpProblem("all_armors", pulp.LpMaximize)

    armor_types = ["head", "chest", "arms", "legs"]

    typed_armor_lists = [[Armor(**armor) for armor in my_armors[armor_type]] for armor_type in armor_types] if my_armors_only else \
        [get_armors(armor_type) for armor_type in armor_types]


    armors = sum(typed_armor_lists, [])

    # Only one of each piece
    for armor_list in typed_armor_lists:
        prob += sum([a.var for a in armor_list]) <= 1.0

    # weight within limit
    weight = sum([armor.weight * armor.var for armor in armors])
    prob += weight <= weight_limit

    # maximise weight
    poise = sum([armor.poise * armor.var for armor in armors])
    prob += poise

    prob.solve(pulp.PULP_CBC_CMD(msg=0))


    print("Name - Poise - Weight ")

    for a in armors:
        if pulp.value(a.var) > 0:
            print(a.name, "-", a.poise, "-", a.weight)

    print("Total - ", pulp.value(poise), "-", pulp.value(weight))

    

In [2]:
solve(weight_limit=21.6, my_armors_only=True)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3q/_dxdmqw96_x981j828nlpw9r0000gn/T/3956f1ce23ca455c81b6d5cb9b8c7abf-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3q/_dxdmqw96_x981j828nlpw9r0000gn/T/3956f1ce23ca455c81b6d5cb9b8c7abf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 206 RHS
At line 212 BOUNDS
At line 252 ENDATA
Problem MODEL has 5 rows, 39 columns and 78 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 36.2609 - 0.00 seconds
Cgl0008I 4 inequality constraints converted to equality constraints
Cgl0005I 4 SOS with 33 members
Cgl0004I processed model has 5 rows, 33 columns (33 integer (33 of which binary)) and 62 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial 